In [1]:
!pip install transformers

In [2]:
from transformers import AutoModelForTokenClassification,AutoTokenizer,pipeline
model = AutoModelForTokenClassification.from_pretrained('uer/roberta-base-finetuned-cluener2020-chinese')
tokenizer = AutoTokenizer.from_pretrained('uer/roberta-base-finetuned-cluener2020-chinese', model_max_length=512)
ner = pipeline('ner', model=model, tokenizer=tokenizer)

config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/407M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/295 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [12]:
def ner_by_cluener(title, tag_type):
  ner_result = ner(title)
  single_tag_str = ""
  tag_list = []
  for char_dic in ner_result:
    # b_or_i, current_entity_type = char_dic['entity'].split("-")
    current_entity_type = char_dic['entity'].split("-")[1]
    # Start a new entity
    if single_tag_str == "" and (char_dic['entity'] == f"B-{tag_type}" or char_dic['entity'] == f"I-{tag_type}"):
      single_tag_str = char_dic['word']
    # Concatenate the entity
    elif char_dic['entity'] == f"I-{tag_type}":
      single_tag_str+=char_dic['word']
    # B-tag type follows another B-tag type
    elif char_dic['entity'] == f"B-{tag_type}":
      tag_list.append(single_tag_str)
      single_tag_str = char_dic['word']
    # B-tag type ended by a non-type type
    elif single_tag_str!= "" and tag_type!= current_entity_type:
      tag_list.append(single_tag_str)
      single_tag_str = ""
# The last captured entity
  if single_tag_str != "":
    tag_list.append(single_tag_str)
  # tag_list = [i for i in tag_list if len(i)>1]
  tag_list = list(set(tag_list))
  print(f"{tag_type} - {tag_list}")
  return ";".join(tag_list)

In [14]:
import csv
import os

# 注意：用户需要且仅需要设置以下的变量mountedGdriveWorkspace, fileName和tags
#   - mountedGdriveWorkspace：输入、输出数据的文件夹路径
#   - fileName：待tag文本文件名，位于mountedGdriveWorkspace路径下，且已经简转繁
#   - tags：标记标签

# 如果目录为Google Drive，那么首先需要mount到colab。
# 以下YOUR_FULL_GOOGLEDRIVE_FILE_SYSTEM为使用者的Google Drive
#
mountedGdriveWorkspace = "/content/drive/MyDrive/YOUR_FULL_GOOGLEDRIVE_FILE_SYSTEM"

# 待tag文本文件名，位于mountedGdriveWorkspace路径下，且已经简转繁
#
fileName = "input.txt"

# 需要挖掘的信息的数组，可为name, position, address等tag的任意组合
# 例一：标记name，tags = ["name"]
# 例二：标记name和position，tags = ["name", "position"]
# 例三：标记name，position和address，tags = ["name", "position", "address"]
tags = ["name", "position", "address"]

# 另：
#   1. 输出文件可在mountedGdriveWorkspace目录下找到，名为output_XXX.csv, XXX=不带扩展名的fileName
#   2. 临时输出文件夹亦可在mountedGdriveWorkspace目录下找到，名为temp_output_XXX, XXX=不带扩展名的fileName
#

print("Start tagging...")

# output folder is on your google drive
#
mainName = fileName.split(".")[0]
tempOutputDir = f"{mountedGdriveWorkspace}/temp_output_{mainName}_{tags}"

output = []
count = 0

if not os.path.exists(tempOutputDir):
    os.mkdir(tempOutputDir)

with open(f"{mountedGdriveWorkspace}/{fileName}", 'r', encoding='utf-8') as f_reader:
  input_data = [i.rstrip('\n') for i in f_reader.readlines()]
  count_max = len(input_data)
  for title in input_data:
    if count%2000 == 0 and count != 0:
        print("{:.2f}% finished...".format(count/count_max*100))
        with open(f"{tempOutputDir}/{mainName}_{tags}_output_{count}.csv", "w", encoding='utf-8', newline='') as f_writer:
            csv_writer = csv.writer(f_writer)
            csv_writer.writerows(output)
            output = []

    lineArr = [title]
    for tag in tags:
        lineArr.append(ner_by_cluener(title, tag));

    output.append(lineArr)
    count+=1

if output != []:
  with open(f"{tempOutputDir}/{mainName}_{tags}_output_{count}.csv", "w", encoding='utf-8', newline='') as f_writer:
    csv_writer = csv.writer(f_writer)
    csv_writer.writerows(output)

Start tagging...
name - []
position - []
address - []
name - []
position - []
address - []
name - ['舟', '楊子圓']
position - []
address - []
name - ['簡敬軒']
position - []
address - []
name - []
position - []
address - []
name - ['陳雲泉', '廣信']
position - ['醫博士']
address - []
name - ['史']
position - ['太史']
address - []
name - ['劉禹錫']
position - []
address - []
name - []
position - []
address - []
name - ['陳常庵']
position - []
address - []
name - ['陳摶']
position - []
address - []
name - []
position - []
address - []
name - ['周卿']
position - ['知府']
address - []
name - ['趙達夫']
position - []
address - []
name - []
position - []
address - []
name - ['趙達夫']
position - []
address - []
name - []
position - []
address - []
name - []
position - []
address - []
name - ['王仙卿']
position - []
address - []
name - []
position - []
address - ['平江府']
name - []
position - []
address - []
name - ['將仕']
position - []
address - []
name - []
position - []
address - []
name - ['李仁甫']
position - []
address - []
name -

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [15]:
import csv
from glob import glob
import os

print("Combining output...")
count = 0
output = []
with open(f"{mountedGdriveWorkspace}/output_{mainName}_{tags}.csv", "w", encoding='utf-8', newline='') as f_writer:
    csv_writer = csv.writer(f_writer)
    for filename in glob(f"{tempOutputDir}/*.csv"):
        with open(filename, newline='') as f:
            reader = csv.reader(f)
            for row in reader:
                csv_writer.writerow(row)

print("Removing temp data...")
cmd = f"rm -r {tempOutputDir}/*.csv"
# remove all csv files in the temp folder,
# comment out the next line if you want to keep them
#
os.system(cmd);

print("Done!")

Combining output...
Removing temp data...
Done!
